# 一个简单的多线程爬虫

In [16]:
# -*-coding:utf8-*-

from multiprocessing.dummy import Pool as ThreadPool
import requests
import time

def get_source(url):
    return requests.get(url)

urls = []

for i in range(1, 21):
    newpage = "https://tieba.baidu.com/p/3522395718?pn=" + str(i)
    urls.append(newpage)
    
time1 = time.time()
for i in urls:
    print(i)
    get_source(i)
time2 = time.time()
print("单线程耗时：", time2 - time1)

pool = ThreadPool(4)
time3 = time.time()
requestse = pool.map(get_source, urls)
pool.close()
pool.join()
time4 = time.time()
print("多线程耗时：", time4 - time3)

多线程耗时： 4.215241193771362


# 多线程 + XPath爬百度贴吧

In [17]:
from lxml import etree
from multiprocessing.dummy import Pool as ThreadPool
import requests
import json


def to_write(content_dict):
    f.writelines("回帖人：" + str(content_dict["author"]) + "\n")
    f.writelines("回帖内容：" + str(content_dict["content"]) + "\n")
    f.writelines("回帖时间：" + str(content_dict["reply_time"]) + "\n")

def spider(url):
    html = requests.get(url)
    selector = etree.HTML(html.text)
    content_field = selector.xpath('//*[@class="l_post j_l_post l_post_bright  "]')
    item = {}
    for each in content_field:
        reply_info = json.loads(each.xpath("@data-field")[0])
        author = reply_info["author"]["user_name"]
        content = each.xpath(
            'div[@class="d_post_content_main"]/div/cc/div[@class="d_post_content j_d_post_content  clearfix"]')[0].xpath('string(.)')
        reply_time = reply_info["content"]["date"]

        item["author"] = author
        item["content"] = content.strip()
        item["reply_time"] = reply_time
        to_write(item)

pool = ThreadPool(4)
f = open("content.txt", "a", encoding="utf-8")
page = []
for i in range(1, 11):
    new_page = 'https://tieba.baidu.com/p/3522395718?pn=' + str(i)
    page.append(new_page)
results = pool.map(spider, page)
pool.close()
pool.join()
f.close()